In [1]:
!pip install librosa
!pip install pydub
!pip install fastai==1.0.58

Keyring is skipped due to an exception: org.freedesktop.DBus.Error.InvalidFileContent: D-Bus library appears to be incorrectly set up: see the manual page for dbus-uuidgen to correct this issue. (Failed to open "/var/lib/dbus/machine-id": No such file or directory; UUID file '/etc/machine-id' should contain a hex string of length 32, not length 0, with no other text)
Defaulting to user installation because normal site-packages is not writeable
Keyring is skipped due to an exception: org.freedesktop.DBus.Error.InvalidFileContent: D-Bus library appears to be incorrectly set up: see the manual page for dbus-uuidgen to correct this issue. (Failed to open "/var/lib/dbus/machine-id": No such file or directory; UUID file '/etc/machine-id' should contain a hex string of length 32, not length 0, with no other text)
Defaulting to user installation because normal site-packages is not writeable
Keyring is skipped due to an exception: org.freedesktop.DBus.Error.InvalidFileContent: D-Bus library app

In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path
from pydub import AudioSegment
from pydub.utils import make_chunks
import random
from scipy.io import wavfile

/home/marcel/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [3]:
# processing parameter
windowsize = 256  # the length of the windowing segments in spectrogram
chunksize = 2000  # length of time window in milliseconds
channel = 0  # channel 0 or 1 from the wav-file for spectrogram

NOverlap = 0
dt = 0.0005
Fs = int(1.0 / dt)  # the sampling frequency

In [4]:
# Create wav files chunks
def create_chunks(path, filename, tag, chunksize=10000):
    """
        This method splits the given wav file in wav files of chunk seconds size

    :param filename: Original wav file
    :param label: Label for new file
    :param chunksize: Chunk size (default 10 sec.)
    :return: List of splitted files
    """
    print('=> create_chunks [dir %s, filename %s, chunksize %s]' % (path, filename, chunksize))

    # list of files
    files = []
    # incoming file as loaded wav
    myaudio = AudioSegment.from_file(os.path.join(path,filename), "wav")
    # Make chunks of length of specified chunk size (in ms)
    chunks = make_chunks(myaudio, chunksize)

    img_name = os.path.split(filename)[1].split('.')[0]
    directory = os.path.join(path,'chunks', tag)

    if not os.path.exists(directory):
        os.makedirs(directory)

    # Export all of the individual chunks as wav files
    for i, chunk in enumerate(chunks):
        chunk_name = os.path.join(directory, '{0}-{1}.wav'.format(img_name, i))
        print('\t writing . %s' % chunk_name)
        chunk.export(chunk_name, format="wav")
        files.append(chunk_name)
    return files

In [5]:
# reade wav file from disk
def read_wav(fn):
    # Read file and get sampling freq [ usually 44100 Hz ]  and sound object
    samplefeq, mySound = wavfile.read(fn)
    soundOneChannel = mySound[:]
    return soundOneChannel

In [6]:
files = create_chunks("./data/sound", "data_ok.wav", "ok")

=> create_chunks [dir ./data/sound, filename data_ok.wav, chunksize 10000]
	 writing . ./data/sound/chunks/ok/data_ok-0.wav
	 writing . ./data/sound/chunks/ok/data_ok-1.wav
	 writing . ./data/sound/chunks/ok/data_ok-2.wav
	 writing . ./data/sound/chunks/ok/data_ok-3.wav
	 writing . ./data/sound/chunks/ok/data_ok-4.wav
	 writing . ./data/sound/chunks/ok/data_ok-5.wav
	 writing . ./data/sound/chunks/ok/data_ok-6.wav
	 writing . ./data/sound/chunks/ok/data_ok-7.wav
	 writing . ./data/sound/chunks/ok/data_ok-8.wav
	 writing . ./data/sound/chunks/ok/data_ok-9.wav
	 writing . ./data/sound/chunks/ok/data_ok-10.wav
	 writing . ./data/sound/chunks/ok/data_ok-11.wav
	 writing . ./data/sound/chunks/ok/data_ok-12.wav
	 writing . ./data/sound/chunks/ok/data_ok-13.wav
	 writing . ./data/sound/chunks/ok/data_ok-14.wav
	 writing . ./data/sound/chunks/ok/data_ok-15.wav
	 writing . ./data/sound/chunks/ok/data_ok-16.wav
	 writing . ./data/sound/chunks/ok/data_ok-17.wav


In [7]:
# create the name we need and creates the directory if it does not exist
def create_spectrogram_name(fn, path, tag):
    _path = os.path.abspath(os.path.join(path, 'train', tag))
    if (not os.path.exists(_path)):
        os.makedirs(_path)
    return os.path.join(_path, fn)


# save the given data as spectrogram to disc for given filename
# https://stackoverflow.com/questions/47147146/save-spectrogram-only-content-without-axes-or-anything-else-to-a-file-using-m
def create_spetrogram(path, fn, tag):
    """
        Creates a spectrogram and store it as image.

    :param path: Root data path
    :param fn:  Filename of wav to be processed
    :param format: Spectrogram format
    :return:
    """
    data = read_wav(fn)

    fig, ax = plt.subplots(1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.axis('off')

    filename = os.path.split( fn )[1].split( '.' )[0]
    fn = create_spectrogram_name( filename, path, tag )

    data = np.array(data).astype('float')
    S = librosa.feature.melspectrogram(y=data, sr=Fs)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))

    print('\t create spectrogram . %s' % os.path.abspath(fn))
    fig.savefig(os.path.abspath(fn), dpi=300, frameon='false')
    plt.close()

In [8]:
for fn in files:
    create_spetrogram( "./data", fn, "ok" )

	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-0


/home/marcel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "frameon" which is no longer supported as of 3.3 and will become an error two minor releases later


	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-1
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-2
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-3
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-4
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-5
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-6
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-7
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-8
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-9
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/train/ok/data_ok-10
	 create spectrogra

In [9]:
files = create_chunks("./data/sound", "data_kavitation.wav", "kavitation")
for fn in files:
    create_spetrogram( "./data/sound", fn, "kavitation" )

=> create_chunks [dir ./data/sound, filename data_kavitation.wav, chunksize 10000]
	 writing . ./data/sound/chunks/kavitation/data_kavitation-0.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-1.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-2.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-3.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-4.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-5.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-6.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-7.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-8.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-9.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-10.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-11.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-12.wav
	 writing . ./data/sound/chunks/kavitation/data_kavitation-13.wav
	 w

/home/marcel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "frameon" which is no longer supported as of 3.3 and will become an error two minor releases later


	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-1
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-2
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-3
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-4
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-5
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-6
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-7
	 create spectrogram . /mnt/c/Users/tima268/repos/inf-ks/inf-ks-notebooks/data/sound/train/kavitation/data_kavitation-8
	 create spectrogram . /mnt/c/Users/tima


# From here start your own classifier!